**Dataset credits and copyrights:**

InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  
  title     = {Learning Word Vectors for Sentiment Analysis},
  
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  
  month     = {June},
  
  year      = {2011},
  
  address   = {Portland, Oregon, USA},
  
  publisher = {Association for Computational Linguistics},
  
  pages     = {142--150},
  
  url       = {http://www.aclweb.org/anthology/P11-1015}
}


References

Potts, Christopher. 2011. On the negativity of negation. In Nan Li and
David Lutz, eds., Proceedings of Semantics and Linguistic Theory 20,
636-659.


### 1.a.1. Downloading dataset,extracting compressed files and moving into the folder containg data

In [0]:
!rm -rf datalab
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!ls
!uncompress aclImdb_v1.tar.gz
!tar -xvf aclImdb_v1.tar
!rm aclImdb_v1.tar
!ls
!pip install stop-words
import nltk
nltk.download('popular')
!pip install -U gensim

testing GPU:

In [0]:
# check whether we connected to GPU or not
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

directories tree:

In [0]:
!ls aclImdb
!ls aclImdb/train

### **Reading files containing reviews**

getting text files from folders:

In [0]:
import glob
'''
function to read all text file names in a given folder
@param  path: given in this form '/path/'
'''
def txt_file_names_list(path):
    txt_files=glob.glob('.'+path+'*.txt')
    return txt_files
  
#testing that function actually works
text_files=txt_file_names_list('/aclImdb/train/pos/')
print(len(text_files))
print(text_files)

In [0]:
#made lists of names of files containing reviews
#tested
pos_train_txt_names = txt_file_names_list('/aclImdb/train/pos/')
neg_train_txt_names = txt_file_names_list('/aclImdb/train/neg/')
pos_test_txt_names = txt_file_names_list('/aclImdb/test/pos/')
neg_test_txt_names = txt_file_names_list('/aclImdb/test/neg/')



reading the files to get the reviews' text:

In [0]:
'''
Function
Given an array of strings of paths(names) of files to be read 
reads each file and saves all text in a string list

@param text_file_names     list of text file names and paths to them
@return text_files         list of contents of given text files
'''
#tested and it works
def capture_text_from_files(text_file_paths,number_of_files):
  text_files=[]
  for fileNo in range(len(text_file_paths)):
      f = open(text_file_paths[fileNo], 'r')
      lines = f.readlines()
      text=""
      f.close()
      for lineNo in range(len(lines)):
        text+=" "+lines[lineNo]
      text_files.append(text)
  return text_files
  
#testing 
pos_reviews_train=capture_text_from_files(pos_train_txt_names,len(pos_train_txt_names) )
print(len(pos_reviews_train))
print(pos_train_txt_names[0])
print(pos_reviews_train[1])

  

In [0]:
#getting reviews text into lists to preprocess :
pos_reviews_train_list=capture_text_from_files(pos_train_txt_names,len(pos_train_txt_names) )
neg_reviews_train_list=capture_text_from_files(neg_train_txt_names,len(neg_train_txt_names) )
pos_reviews_test_list=capture_text_from_files(pos_test_txt_names,len(pos_test_txt_names) )
neg_reviews_test_list=capture_text_from_files(neg_test_txt_names,len(pos_train_txt_names) )


### Text Preprocessing

removing html tags

In [0]:
#removing html tags from reviews not to confuse classifiers

import nltk
from bs4 import BeautifulSoup
'''
Function to clear all html tags from all text in a list of strings

@param text_list : list of strings targeted to clean
'''
def clean_html_from_list(text_list):

  for i in range(len(text_list)):
    text_list[i]=BeautifulSoup(text_list[i],"html5lib").get_text()
    
  return text_list



In [0]:
#using the function on data
clean_html_from_list(pos_reviews_train_list)
clean_html_from_list(neg_reviews_train_list)
clean_html_from_list(pos_reviews_test_list)
clean_html_from_list(neg_reviews_test_list)

### spell checker : (still no good ones in libraries and will use a naive one or implement our own but not necessary )
didn't do it to prevent computing overload by hunch it won't improve accuracy by much

In [0]:
'''
#TODO
!pip install -U textblob
!pip install -U autocorrect
from textblob import TextBlob
from autocorrect import spell
'''

In [0]:
'''
blob = TextBlob('tihs assignmt si on itss tim excpt fr midtrms')
print("blob's:")
print(blob.correct())

x='tihs assignmt si on itss tim excpt fr midtrms'
print("autocorrect:")
print(spell(x))

#blob better and faster

'''

tokenization:

In [0]:
from nltk.tokenize import word_tokenize
def tokenize_list(list_words):
  tokens=[]
  for review in list_words:
    tokens.append(word_tokenize(review))
  return tokens

In [0]:
import nltk
nltk.download('punkt')
pos_reviews_train_list=tokenize_list(pos_reviews_train_list)
neg_reviews_train_list=tokenize_list(neg_reviews_train_list)
pos_reviews_test_list=tokenize_list(pos_reviews_test_list)
neg_reviews_test_list=tokenize_list(neg_reviews_test_list)

In [0]:

pos_train_tokens=pos_reviews_train_list
neg_train_tokens=neg_reviews_train_list
pos_test_tokens=pos_reviews_test_list
neg_test_tokens=neg_reviews_test_list

## stop words removal (is,are,the,this ....etc) :


removing any stop words (written in capital or small letters) and removing punctuations

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from functools import reduce
import operator


stop_words = list(get_stop_words('en')) 

def remove_stopwords(review):
  filtered_words=[]
  for tokens in review:
    filtered_word=[]
    filtered_word = [word for word in tokens if word.lower() not in stop_words]
    filtered_words.append(filtered_word)
    
  return(filtered_words)
def remove_punctuations(review):
  filtered_words=[]
  filtered=[]
  for tokens in review:
    filtered_word=[]
    for token in tokens:
        if (token.isalpha()== False):
          filtered_word.append(token)
    filtered_words=[x.lower() for x in tokens if x not in  filtered_word]
    filtered.append(filtered_words)
          
    
  return(filtered)

x=remove_stopwords([['the','love',';','.','an','is','hi','is','the','big','are','have','they','efvfdv','I','There','am','i','IS','They'],['big','task']])
print(x)
X=remove_punctuations(x)
print(X)


In [0]:
pos_train_tokens=remove_stopwords(pos_reviews_train_list)
pos_train_tokens=remove_punctuations(pos_train_tokens)
print(pos_train_tokens[0])
neg_train_tokens=remove_stopwords(neg_reviews_train_list)
neg_train_tokens=remove_punctuations(neg_train_tokens)

pos_test_tokens=remove_stopwords(pos_reviews_test_list)
pos_test_tokens=remove_punctuations(pos_test_tokens)

neg_test_tokens=remove_stopwords(neg_reviews_test_list)
neg_test_tokens=remove_punctuations(neg_test_tokens)



**Lemmatization**

better than stemming

In [0]:
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
      


def lemmatize_list(list_of_lists_of_tokens):
  lemmatizer = WordNetLemmatizer()
  new_list=[]
  for review in list_of_lists_of_tokens:
    new_review=[]
    for token in review:
      
      if(token not in ["movie","film","films","movies","will","can"]):
        new_token=x=lemmatizer.lemmatize(token, get_wordnet_pos(nltk.pos_tag([token])[0][1]))
        new_review.append(new_token)
        
    new_list.append(new_review)
  return new_list

In [0]:
print(pos_train_tokens[0])

In [0]:
'''
pos_train_tokens=lemmatize_list(pos_train_tokens)
print("lemmatizing 1st")
neg_train_tokens=lemmatize_list(neg_train_tokens)
print("lemmatizing 2nd")
pos_test_tokens=lemmatize_list(pos_test_tokens)
print("lemmatizing 3rd")
neg_test_tokens=lemmatize_list(neg_test_tokens)
print("lemmatizing 4th")
'''

In [0]:
'''
from nltk import word_tokenize, pos_tag, ne_chunk, tree2conlltags
nltk.download('popular')
# names and locations removing routine
# still needs further improvements
def remove_name_loc(text):
    one=[]
    filter=[]
    for tokens in text:

      ne_tag = tree2conlltags(ne_chunk(pos_tag(tokens)))

      i=0
      for token in tokens:

        if str(ne_tag[i][2])== 'B-PERSON':
          pass
        elif str(ne_tag[i][2])== 'B-GPE':
          pass
        elif str(ne_tag[i][2])== 'B-ORGANIZATION':
          pass
        else:
          one.append(token)
        i+=1

      filter+=[one[:]]
      del one[:]
    return filter
 '''   

In [0]:

'''
pos_train_tokens=remove_name_loc(pos_train_tokens)
neg_train_tokens=remove_name_loc(neg_train_tokens)
pos_test_tokens=remove_name_loc(pos_test_tokens)
neg_test_tokens=remove_name_loc(neg_test_tokens)
'''


In [0]:
# ZYAD STARTED HERE
'''
from nltk import word_tokenize, pos_tag, ne_chunk, tree2conlltags
 
sentence = "Mark and Kareem are working at Canada."
 
ne_tree = ne_chunk(pos_tag(word_tokenize(sentence)))
 
iob_tagged = tree2conlltags(ne_tree)
print(iob_tagged)

# names and locations removing routine
# still needs further improvements
def remove_name_loc(text):
    filtered_words=[]
    for tokens in text:
      ne_tag = tree2conlltags(ne_chunk(pos_tag(tokens)))
      i=0
      for token in tokens:
        print(token,ne_tag[i][2])
        if str(ne_tag[i][2])== 'B-PERSON':
          pass
        elif str(ne_tag[i][2])== 'B-GPE':
          pass
        else:
          filtered_words.append(token)
        i+=1
    print(filtered_words)

remove_name_loc([['Canada','Adams','nice'],['yes']])'''

stemming worked badly so removed it this stemmer is better than porter stemmer yet it reduces accuracy and without it the classifying accuracy is better even without lemmatization

In [0]:
'''
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def stem_list(list_of_lists_of_tokens):
  new_list=[]
  for review in list_of_lists_of_tokens:
    new_review=[]
    for token in review:
      if(token not in ["movie","film","films","movies","will"]):
        new_token=stemmer.stem(token)
        new_review.append(new_token)
    new_list.append(new_review)
  return new_list

trial=[ ["list","word","playing","movie"],["film","production"],["tokenize","stem"]]
trial=stem_list(trial)
print(trial)

'''

In [0]:
'''
#Kareem's temporary stemming
pos_trian_tokens=stem_list(pos_train_tokens)
print("stemmed 1st")
neg_train_tokens=stem_list(neg_train_tokens)
print("stemmed 2nd")
pos_test_tokens=stem_list(pos_test_tokens)
print("stemmed 3rd")
neg_test_tokens=stem_list(neg_test_tokens)
print("4th")
'''

Mahinour's trial for ** Inverse Document Frequency (TF – IDF)** - Done , only def in a function remains




In [0]:
#trial is reviews of combined tokens returned from tokenized form
trial=[]
for j in range (len(pos_train_tokens)):
  trial1=""
  for i in range(len(pos_train_tokens[j])):
    trial1+=pos_train_tokens[j][i]+' '
  trial.append(trial1)
  
trial2=[]
for j in range (len(neg_train_tokens)):
  trial1=""
  for i in range(len(neg_train_tokens[j])):
    trial1+=neg_train_tokens[j][i]+' '
  trial2.append(trial1)
  
  
trial3=[]
for j in range (len(neg_test_tokens)):
  trial1=""
  for i in range(len(neg_test_tokens[j])):
    trial1+=neg_test_tokens[j][i]+' '
  trial3.append(trial1)
  
trial4=[]
for j in range (len(pos_test_tokens)):
  trial1=""
  for i in range(len(pos_test_tokens[j])):
    trial1+=pos_test_tokens[j][i]+' '
  trial4.append(trial1)  

print("done")
    

In [0]:
'''q=[]
q.extend(neg_test_tokens)
q.extend(pos_test_tokens)
'''

In [0]:
#train data is list of positive and negative reviews full after preprocessing
train_data=[]
test_data=[]
train_data.extend(trial)
train_data.extend(trial2)
test_data.extend(trial3)
test_data.extend(trial4)

# Doc2Vec

In [0]:
'''
!pip install -U gensim
from gensim.models import Doc2Vec

from gensim.models import doc2vec
from collections import namedtuple
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

# Transform data (you can add more data preprocessing steps) 

docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(train_data):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))
    
    
model = doc2vec.Doc2Vec(docs , dm_concat=1, vector_size = 100, window = 300, negative=5, hs=0, min_count = 2, workers = cores)

print(docs)
'''


## MAXIMUM ACCURACY ATTAINED TIL NOW FOR DOC2VEC IS 73% USING LOG. REGRESSION CLASSIFIER :S

In [0]:
'''
from gensim.models import doc2vec
from collections import namedtuple
import random

# Transform data (you can add more data preprocessing steps) 
assert doc2vec.FAST_VERSION >-1
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(train_data):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))

# Train model (set min_count = 1, if you want the model to work with the provided example data set)

#model = doc2vec.Doc2Vec(docs, vector_size = 100, window = 20, alpha=0.025, min_alpha=0.025, min_count = 1, workers = 8)
model= doc2vec.Doc2Vec(docs,min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)
for epoch in range(15):
    random.shuffle(docs)
    model.train(docs, total_examples=model.corpus_count, epochs=model.epochs)
    #model.alpha -= 0.002 # decrease the learning rate
    #model.min_alpha = model.alpha # fix the learning rate, no deca
    #model.train(docs)
    '''


In [0]:
#model.most_similar('comedy')

In [0]:
# http://linanqiu.github.io/2015/10/07/word2vec-sentiment/
# https://github.com/linanqiu/word2vec-sentiments/blob/master/word2vec-sentiment.ipynb
# https://medium.com/@mishra.thedeepak/doc2vec-in-a-simple-way-fa80bfe81104

# <<zyad>> you may open links for illustration about the edits i "tried" to make


In [0]:
'''
# ZYAD STARTED HERE

# not sure whether this step is valid or not:
# i'm trying to generate test reviews vectors by inference using our training model
# not sure about the concept but it seems that inferring process is like finding similarity of certain test vector against the training model
!pip install -U numpy
import numpy as np

train_vectors= np.zeros((25000, 100))
test_vectors= np.zeros((25000, 100))
for i in range(25000):
  label = 'test_%s' %i
  train_vectors[i]=model[i]
  test_vectors[i]= model.infer_vector(word_tokenize(test_data[i]))
  '''

# Tf-IDF and count vectorizer

In [0]:
##count #commented to avoid confusion regarding "test_vectors" & "train_vectors" of mine
'''
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)
'''

In [0]:
## tf-IDf #commented to avoid confusion regarding "test_vectors" & "train_vectors" of mine

from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
obj = TfidfVectorizer(ngram_range=(1,2),stop_words='english')
train_vectors = obj.fit_transform(train_data)
test_vectors = obj.transform(test_data)



In [0]:
train_label_pos = [0] * 12500
train_label_neg = [1] * 12500
test_label_pos = [0] * 12500
test_label_neg = [1] * 12500
train_labels=[]
test_labels=[]
train_labels.extend(train_label_pos)
train_labels.extend(train_label_neg)
test_labels.extend(train_label_neg)
test_labels.extend(train_label_pos)
#print(test_labels)

In [0]:
train_vectors10=train_vectors[11250:13750]
train_labels10=train_labels[0:1250]
train_labels10.extend(train_labels[-1250:])
train_vectors90=train_vectors[1250:-1250]
train_labels90=train_labels[1250:-1250]


In [0]:
print(train_vectors10.shape)
print(len(train_labels10))

**Tuning**

we take 10% of data to extract best parameters for each classifier
the function grid_search_wrapper returns the classifier loaded with best parameters that give best results

and then fit data (90%) using this classifier

In [0]:
'''
function takes precision score type like accuracy,precision,recall ...etc and 
returns classifier with best parameters 
the function has other parameters like clf , train vectors and labels ...etc
but they are either hard coded inside or use global variables that change on 
each classifier cell so it is not generic and cannot be used outside this notebook

'''

def grid_search_wrapper(refit_score='precision_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    
    skf = StratifiedKFold(n_splits=10)
    '''
    GridSearchCV 
    clf is a global variable outside function  and set before calling it 
    
    param_grid is the parameters values possible also global variables 
    
    scorers always is the accuracy score here
    
    
    '''
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                           cv=skf, return_train_score=True)
    grid_search.fit(train_vectors10,train_labels10)
    print("fit completed")
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    print("best params completed")
    print(grid_search.best_estimator_)
    print("best params completed")
    
    return grid_search.best_estimator_

In [0]:
##logistic Regression
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression(C=1000)
logreg.fit(train_vectors, train_labels)
prediction = logreg.predict(test_vectors)
# accuracy = getAccuracy(train_labels, prediction)
# print('Accuracy: ' + repr(accuracy) + '%')

print(logreg.score(test_vectors,test_labels))

In [0]:
#tuning log reg
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression(C=1000)
logreg.fit(train_vectors, train_labels)
prediction = logreg.predict(test_vectors)
# accuracy = getAccuracy(train_labels, prediction)
# print('Accuracy: ' + repr(accuracy) + '%')

print(logreg.score(test_vectors,test_labels))

In [0]:
'''
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

clf = linear_model.LogisticRegression()

param_grid = {
  'C' :[0.1,100,1000,10000]

}
scorers = {
    'accuracy_score': make_scorer(accuracy_score)
}


grid_search_clf = grid_search_wrapper(refit_score='accuracy_score')
grid_search_clf.fit(train_vectors, train_labels)
print(grid_search_clf.score(test_vectors,test_labels))
'''


In [0]:

### Naive-bayes
from sklearn.naive_bayes import MultinomialNB
clfs = MultinomialNB()
clfs.fit(train_vectors, train_labels)
print(clfs.score(test_vectors,test_labels))


In [0]:
##Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

clf = RandomForestClassifier(n_jobs=-1)

param_grid = {
   # 'min_samples_split': [1000,10000], 
    #'n_estimators' : [100,300],
    'n_estimators' : [300,500],
    #'max_depth': [10,30],
   # 'max_features': [10,20]
}
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}

grid_search_clf = grid_search_wrapper(refit_score='accuracy_score')
grid_search_clf.fit(train_vectors, train_labels)
print(grid_search_clf.score(test_vectors,test_labels))

In [0]:
##Adaboost
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1,
                         random_state=0)
model = clf.fit(train_vectors, train_labels)
prediction = clf.predict(test_vectors)
print(clf.score(test_vectors,test_labels))

In [0]:
##Adaboost tuned
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

clf = AdaBoostClassifier()

param_grid = {
 
    #'n_estimators' : [50,100,300,400],
    'n_estimators' : [50,400],
    #'learning_rate': [0.01,0.1,0.5,1],
    'learning_rate': [0.5,1],
 
}
scorers = {

    'accuracy_score': make_scorer(accuracy_score)
}

grid_search_clf = grid_search_wrapper(refit_score='accuracy_score')
grid_search_clf.fit(train_vectors, train_labels)
print(grid_search_clf.score(test_vectors,test_labels))


In [0]:
###Decision trees
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_vectors, train_labels)
print(clf.score(test_vectors,test_labels))


In [0]:
##Svm non linear
from sklearn import svm
print("rbf")
model = svm.SVC(kernel='rbf') 
model.fit(train_vectors, train_labels) 
prediction = model.predict(test_vectors)
print(model.score(test_vectors,test_labels))



In [0]:
## Svm tunning
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
scorers = {

    'accuracy_score': make_scorer(accuracy_score)
}
clf = svm.SVC(kernel='rbf') 
grid_search_clf = grid_search_wrapper(refit_score='accuracy_score')
grid_search_clf.fit(train_vectors, train_labels)
print(grid_search_clf.score(test_vectors,test_labels))

In [0]:
'''
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(train_vectors, train_labels)
prediction = knn.predict(test_vectors)
accuracy = getAccuracy(train_labels, prediction)
print('Accuracy: ' + repr(accuracy) + '%')
print(clf.score(test_vectors,test_labels))
'''

Important notes:
when lemmatized on tf idf accuracy decreased by 2% in some
*tf idf rand forests max fepth =2 0.67


tf idf : log reg :87.5%, adaboost 80% and des trees 70% with lemmatization

without lemma and tuning :rand : 54% (100,300 estimators and 10 and 30 other )and adaboost 79.852%

with tuning: random forests : 85.5%


without preprocessing at all expept html removal: NB 85.4% ,log reg 88.6% best c is 1000,adaboost 80.2%,without tunning any
with tuning : 83.6% rand forests

with stop words removal and html only log reg 88.3%
without any prep : log reg 88.5%


without preprocessing but with n grams 1-3 : log reg :88.5%

with html and lemmatization only: 88.624%

with html ,stopwords and lemmatization : 88.3%

